[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SuperbTUM/ModelShare_Development/blob/master/notebooks/AutoML_autogluon.ipynb)

In [1]:
!pip install aimodelshare --upgrade -q
!pip install autogluon

     |████████████████████████████████| 951 kB 13.4 MB/s 
     |████████████████████████████████| 83 kB 783 kB/s 
     |████████████████████████████████| 291 kB 24.5 MB/s 
     |████████████████████████████████| 22.3 MB 63.4 MB/s 
     |████████████████████████████████| 178 kB 51.3 MB/s 
     |████████████████████████████████| 146 kB 15.6 MB/s 
     |████████████████████████████████| 442 kB 27.8 MB/s 
     |████████████████████████████████| 288 kB 51.2 MB/s 
     |████████████████████████████████| 13.1 MB 1.8 MB/s 
     |████████████████████████████████| 308 kB 15.3 MB/s 
     |████████████████████████████████| 96 kB 1.1 MB/s 
     |████████████████████████████████| 4.5 MB 49.8 MB/s 
     |████████████████████████████████| 132 kB 52.3 MB/s 
     |████████████████████████████████| 8.6 MB 25.9 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 53.1 MB/s 
     |████████████████████████████████| 55 kB 183 kB/s 
     |███████████████

In [2]:
import aimodelshare as ai

In [ ]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


Tutorial for text-based dataset

In [3]:
X_train, X_test, y_train_labels, y_test_labels, example_data = ai.import_quickstart_data("covid_tweets")
# Each sample is a sentence
print(y_train_labels[0])


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
fake


In [4]:
import numpy as np
import pandas as pd
# You don't need to explicitly process these sentences
y_train_labels_processed = np.asarray(list(map(lambda x: 0 if x == "fake" else 1, y_train_labels))).reshape(len(X_train), 1)
y_train_labels_processed = pd.DataFrame(y_train_labels_processed, columns=["labels"])
y_test_labels_processed = np.asarray(list(map(lambda x: 0 if x == "fake" else 1, y_test_labels))).reshape(len(X_test), 1)
y_test_labels_processed = pd.DataFrame(y_test_labels_processed, columns=["labels"])

pd_tweets = pd.concat((X_train, y_train_labels_processed), axis=1)
print(pd_tweets.head())

                                               tweet  labels
0  "[T]he label of the popular Lysol already show...       0
1  There were more deaths on the roads of France ...       0
2  250 new cases of #COVID19Nigeria; Plateau-69 F...       1
3  @XanderArmstrong Why was chloroquine described...       0
4  Our new Can Compare 'College' tags make it eas...       1


Training

In [ ]:
# It is advised to turn on GPU for training and testing
from autogluon.text import TextPredictor

predictor = TextPredictor(label="labels", eval_metric="acc", path="./covid_tweets")
predictor.fit(pd_tweets, presets="medium_quality_faster_train", time_limit=60)

INFO:pytorch_lightning.utilities.seed:Global seed set to 123


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 46: 'val_acc' reached 0.93548 (best 0.93548), saving model to '/content/covid_tweets/epoch=0-step=46.ckpt' as top 3
INFO:pytorch_lightning.utilities.rank_zero:Time limit reached. Elapsed time is 0:01:05. Signaling Trainer to stop.


Validation: 0it [00:00, ?it/s]

INFO:automm:Start to fuse 2 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Testing

In [ ]:
pd_tweets_test = pd.concat((X_test, y_test_labels_processed), axis=1)
test_score = predictor.evaluate(pd_tweets_test, metrics=["acc", "f1"])
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.9435523114355231, 'f1': 0.9457943925233645}


Get the tuned model architecture!

In [ ]:
model = predictor._predictor._model

Save and Load

In [ ]:
loaded_predictor = TextPredictor.load("covid_tweets")
loaded_predictor.predict({"tweet": ["Covid is bad."]})

loaded_predictor.save("./weights")

INFO:automm:Load pretrained checkpoint: covid_tweets/model.ckpt


Predicting: 0it [00:00, ?it/s]

Submit the model

In [ ]:
from aimodelshare import ModelPlayground

myplayground = ModelPlayground(model_type="text", 
                classification=True, 
                private=False)
myplayground.playground_url = 'https://heo6qayusj.execute-api.us-east-2.amazonaws.com/prod/m'

mycompetition = ai.Competition(myplayground.playground_url)

# apiurl = myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m
set_credentials(apiurl='https://heo6qayusj.execute-api.us-east-2.amazonaws.com/prod/m')

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
from autogluon.text.text_prediction.presets import list_text_presets
list_text_presets(verbose=True)

{'default': {'model.hf_text.checkpoint_name': 'google/electra-base-discriminator'},
 'medium_quality_faster_train': {'model.hf_text.checkpoint_name': 'google/electra-small-discriminator',
  'optimization.learning_rate': 0.0004},
 'high_quality': {'model.hf_text.checkpoint_name': 'google/electra-base-discriminator'},
 'best_quality': {'model.hf_text.checkpoint_name': 'microsoft/deberta-v3-base',
  'env.per_gpu_batch_size': 2},
 'multilingual': {'model.hf_text.checkpoint_name': 'microsoft/mdeberta-v3-base',
  'optimization.top_k': 1,
  'env.precision': 'bf16',
  'env.per_gpu_batch_size': 4}}

You can get the model detail here {'model.hf_text.checkpoint_name': 'google/electra-small-discriminator'}

In [ ]:
# load weights
import torch

weights = torch.load(predictor.path + "/model.ckpt")["state_dict"]
model.load_state_dict(weights, strict=False)

model

HFAutoModelForTextPrediction(
  (model): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_featur

In [ ]:
# data processing
import pickle

with open(predictor.path + "/data_processors.pkl", "rb") as f:
    processor_data = pickle.load(f)
f.close()
processor_text = processor_data["text"] # class
processor_label = processor_data["label"] # class

def preprocessor(pd_data):
    pd_data = processor_text(pd_data)
    return pd_data

import aimodelshare as ai
ai.export_preprocessor(preprocessor,"")
prep=ai.import_preprocessor("preprocessor.zip")


Your preprocessor is now saved to 'preprocessor.zip'


Some problems with transforming huggingface model to onnx model

In [ ]:
# Save pytorch model to local ONNX file
!pip install pyspark

from aimodelshare.aimsonnx import model_to_onnx

# Will send you an error 
"""
onnx_model = model_to_onnx(model, framework=predictor.backend,
              transfer_learning=False,
              deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())
"""


Try the huggingface method instead

In [ ]:
import os

os.path.exists("/content/covid_tweets/model.ckpt")

True

In [ ]:
!wget https://huggingface.co/google/electra-small-discriminator/raw/main/config.json

--2022-11-01 21:56:48--  https://huggingface.co/google/electra-small-discriminator/raw/main/config.json
Resolving huggingface.co (huggingface.co)... 34.227.129.17, 52.3.185.208, 2600:1f18:147f:e850:db19:5c51:ec6e:ddca, ...
Connecting to huggingface.co (huggingface.co)|34.227.129.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 665 [text/plain]
Saving to: ‘config.json.1’

config.json.1       100%[===================>]     665  --.-KB/s    in 0s      

2022-11-01 21:56:48 (123 MB/s) - ‘config.json.1’ saved [665/665]



In [ ]:
!wget https://huggingface.co/google/electra-small-discriminator/raw/main/tokenizer.json

--2022-11-01 22:08:57--  https://huggingface.co/google/electra-small-discriminator/raw/main/tokenizer.json
Resolving huggingface.co (huggingface.co)... 52.3.185.208, 34.227.129.17, 2600:1f18:147f:e850:db19:5c51:ec6e:ddca, ...
Connecting to huggingface.co (huggingface.co)|52.3.185.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466062 (455K) [text/plain]
Saving to: ‘tokenizer.json’

tokenizer.json      100%[===================>] 455.14K  --.-KB/s    in 0.1s    

2022-11-01 22:08:57 (3.06 MB/s) - ‘tokenizer.json’ saved [466062/466062]



In [ ]:
!cp config.json /content/covid_tweets/
!cp tokenizer.json /content/covid_tweets/
torch.save(weights, predictor.path + "/pytorch_model.bin")

In [ ]:
!pip install transformers[onnx]
!mkdir onnx/

# Lack of other constraints
!python -m transformers.onnx --model="/content/covid_tweets/" onnx/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘onnx/’: File exists
2022-11-01 22:09:45.165244: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Some weights of the model checkpoint at /content/covid_tweets/ were not used when initializing ElectraModel: ['model.model.encoder.layer.0.attention.self.query.weight', 'model.model.encoder.layer.1.attention.self.query.weight', 'model.model.encoder.layer.6.output.LayerNorm.bias', 'model.model.encoder.layer.11.attention.output.dense.weight', 'model.model.encoder.layer.2.output.dense.bias', 'model.model.encoder.layer.3.output.dense.bias', 'model.model.encoder.layer.8.output.dense.weight', 'model.model.encoder.layer.6.attention.output.LayerNorm.weight', 'model.model.encoder.layer.10.intermediate.dense.bias', 'model.model.embeddi

Work with automm([multimodal](https://auto.gluon.ai/stable/tutorials/multimodal/beginner_text.html))

In [7]:
from autogluon.multimodal.presets import list_automm_presets
list_automm_presets(verbose=True)

{'default': {'model.hf_text.checkpoint_name': 'google/electra-base-discriminator',
  'model.timm_image.checkpoint_name': 'swin_base_patch4_window7_224'},
 'medium_quality_faster_train': {'model.hf_text.checkpoint_name': 'google/electra-small-discriminator',
  'model.timm_image.checkpoint_name': 'swin_small_patch4_window7_224',
  'optimization.learning_rate': 0.0004},
 'high_quality': {'model.hf_text.checkpoint_name': 'google/electra-base-discriminator',
  'model.timm_image.checkpoint_name': 'swin_base_patch4_window7_224'},
 'best_quality': {'model.hf_text.checkpoint_name': 'microsoft/deberta-v3-base',
  'model.timm_image.checkpoint_name': 'swin_large_patch4_window7_224',
  'env.per_gpu_batch_size': 1},
 'multilingual': {'model.hf_text.checkpoint_name': 'microsoft/mdeberta-v3-base',
  'optimization.top_k': 1,
  'env.precision': 'bf16',
  'env.per_gpu_batch_size': 4},
 'zero_shot': {'model.clip.checkpoint_name': 'openai/clip-vit-large-patch14-336',
  'model.clip.max_text_len': 0,
  'env.

In [9]:
from autogluon.multimodal import MultiModalPredictor

predictor_mm = MultiModalPredictor(label="labels", eval_metric="acc", path="./automm_covid_tweets")
predictor_mm.fit(pd_tweets, presets="medium_quality_faster_train", time_limit=60)

INFO:pytorch_lightning.utilities.seed:Global seed set to 123


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 49: 'val_acc' reached 0.88786 (best 0.93548), saving model to '/content/automm_covid_tweets/epoch=1-step=49.ckpt' as top 3
INFO:automm:Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

In [10]:
pd_tweets_test = pd.concat((X_test, y_test_labels_processed), axis=1)
test_score = predictor_mm.evaluate(pd_tweets_test, metrics=["acc", "f1"])
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.9435523114355231, 'f1': 0.9457943925233645}
